In [1]:
#instaliranje potrebne datoteke
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 33.7 MB/s eta 0:00:00


In [2]:
from Bio import SeqIO
import pandas as pd
import tensorflow as tf
import numpy as np
import random
import keras
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import layers
import math

In [3]:
#spajanje na google drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
#Funkcija za učitavanje FASTA datoteke
def get_sequences_from_fasta(fasta_file):
    sequences = []
    with open(fasta_file, "r") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            sequences.append(str(record.seq))
    return sequences

In [6]:
#funkcija za generiranje skupa podataka za treniranje i validaciju
data_length=500
korak=100
def getData(fasta_file,gff_file_path,podaci):

    sequence = get_sequences_from_fasta(fasta_file)[0]
    print(sequence[:500])

    #ucitavanje gff file-a
    columns = ["seqid", "source", "type", "start", "end", "score", "strand", "phase", "attributes"]
    GFF_DataFrame = pd.read_csv(gff_file_path, sep="\t", comment="#", names=columns)

    #stvaranje vektora koji određuje da li je baza dio gena ili nije
    gene_vector=tf.zeros(shape=len(sequence))
    print(gene_vector[:30])

    for index,row in GFF_DataFrame.iterrows():
        if(row["type"]=="gene"):


            start_position = row["start"]-1
            end_position = row["end"]-1


            mask = tf.concat([
                tf.zeros(start_position),
                tf.ones(end_position - start_position + 1),
                tf.zeros(gene_vector.shape[0] - end_position - 1)
            ], axis=0)


            gene_vector = gene_vector + mask


    gene_vector=gene_vector.numpy()
    for i in range(len(gene_vector)):
        if(gene_vector[i]!=0):
            gene_vector[i]=1
    gene_vector=tf.convert_to_tensor(gene_vector)



    #dijeljenje sekvence na dijelove
    for i in range(math.ceil(len(sequence)/korak)):
        slice_of_seq=sequence[i*korak:i*korak+data_length]
        slice_of_vector=gene_vector[i*korak:i*korak+data_length]

        if(len(slice_of_vector)!=data_length):

            start=(i*korak-len(sequence))
            end=i*korak+data_length-len(sequence)

            #loop around - spaja kraj i pocetak jer je DNA kružna
            slice_of_seq=sequence[start:] + sequence[:end]
            slice_of_vector=tf.concat([gene_vector[start:], gene_vector[:end]], axis=0)

        podaci.append((slice_of_seq,slice_of_vector))
    print("DONE with procesing data")
    return podaci


In [7]:
#dohvaćanje podataka sa google drive-a
import os
data=[]
cwd = os.getcwd()
folder_path = "/content/drive/MyDrive/Test_gene_annotations"
gene_annotations = os.listdir(folder_path)


for bacteria_name in gene_annotations:

    folder_path="/content/drive/MyDrive" +"/Test_gene_annotations/"+bacteria_name+"/data"
    files_in_annotations = os.listdir(folder_path)

    for gca_file in files_in_annotations:


        if(gca_file.startswith("GCA")):
            folder_path="/content/drive/MyDrive"+"/Test_gene_annotations/"+bacteria_name+"/data/"+gca_file
            gca_files = os.listdir(folder_path)

            for file in gca_files:

                if(file.startswith("GCA")):
                    fasta_file="/content/drive/MyDrive/"+"Test_gene_annotations/"+bacteria_name+"/data/"+gca_file+"/"+file

                if(file=="genomic.gff"):
                    gff_file_path="/content/drive/MyDrive/"+"Test_gene_annotations/"+bacteria_name+"/data/"+gca_file+"/"+file
    print("\nProcessing data for "+ bacteria_name+"\n")
    getData(fasta_file,gff_file_path)







Processing data for Pseudomonas_aeruginosa

TTTAAAGAGACCGGCGATTCTAGTGAAATCGAACGGGCAGGTCAATTTCCAACCAGCGATGACGTAATAGATAGATACAAGGAAGTCATTTTTCTTTTAAAGGATAGAAACGGTTAATGCTCTTGGGACGGCGCTTTTCTGTGCATAACTCGATGAAGCCCAGCAATTGCGTGTTTCTCCGGCAGGCAAAAGGTTGTCGAGAACCGGTGTCGAGGCTGTTTCCTTCCTGAGCGAAGCCTGGGGATGAACGAGATGGTTATCCACAGCGGTTTTTTCCACACGGCTGTGCGCAGGGATGTACCCCCTTCAAAGCAAGGGTTATCCACAAAGTCCAGGACGACCGTCCGTCGGCCTGCCTGCTTTTATTAAGGTCTTGATTTGCTTGGGGCCTCAGCGCATCGGCATGTGGATAAGTCCGGCCCGTCCGGCTACAATAGGCGCTTATTTCGTTGTGCCGCCTTTCCAATCTTTGGGGGATATCCGTGTCCGTGGAACTTTGG
        seqid   source    type  start      end score strand phase  \
0  AE004091.2  Genbank  region      1  6264404     .      +     .   
1  AE004091.2  Genbank    gene    483     2027     .      +     .   
2  AE004091.2  Genbank     CDS    483     2027     .      +     0   
3  AE004091.2  Genbank    gene   2056     3159     .      +     .   
4  AE004091.2  Genbank     CDS   2056     3159     .      +     0   

                                       

In [8]:
#kreiranje dataseta za testiranje
test_data = data[:int(len(data))]

(test_seq, test_vector) = zip(*test_data)

test_seq=np.array(test_seq)
test_vector=np.array(test_vector)

test_dataset = tf.data.Dataset.from_tensor_slices((test_seq, test_vector)).batch(32)
print("Veličina skupa za testiranje: " + str(len(test_data)))
print("Duljina dataseta za testiranje: " + str(len(test_dataset)))
print("DONE")

62645
DONE


In [9]:
#custom split funkcija
def custom_split_fn(string_tensor):
    return tf.strings.unicode_split(string_tensor, 'UTF-8')

print(custom_split_fn("ACAAGTTAC"))
print("DONE")

tf.Tensor([b'A' b'C' b'A' b'A' b'G' b'T' b'T' b'A' b'C'], shape=(9,), dtype=string)
DONE


In [10]:
#sloj za vekotrizaciju teksta
text_vectorization= TextVectorization(
    output_mode="int",
    split=custom_split_fn,
    output_sequence_length=data_length,
)
dataset=["ACGTTTGAGTCCAT"]
text_vectorization.adapt(dataset)
print(text_vectorization.get_vocabulary())
print("DONE")

['', '[UNK]', 't', 'g', 'c', 'a']
DONE


In [11]:
#vektoriziranje dataseta
int_test_ds= test_dataset.map(
    lambda x,y: (text_vectorization(x)-2,y),
    num_parallel_calls=4
)
print("DONE")

DONE


In [12]:
for inputs, targets in int_test_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("inputs[0]:", tf.one_hot(inputs[0], depth=4))
  break

inputs.shape: (32, 500)
inputs.dtype: <dtype: 'int64'>
targets.shape: (32, 500)
targets.dtype: <dtype: 'float32'>
inputs[0]: tf.Tensor(
[0 0 0 3 3 3 1 3 1 3 2 2 1 1 2 1 3 0 0 2 0 3 1 0 1 3 3 3 0 2 1 3 3 2 1 1 1
 2 3 1 1 0 2 3 3 0 0 0 2 2 3 3 2 2 3 1 2 1 3 0 1 3 2 1 0 3 3 0 3 1 3 0 3 1
 3 0 3 2 3 3 1 1 3 3 1 0 2 3 0 0 0 0 0 2 0 0 0 0 3 3 3 1 1 3 0 3 1 3 3 3 2
 1 1 0 0 3 3 0 1 2 0 2 0 0 1 1 1 3 2 1 1 2 1 2 0 0 0 0 2 0 1 0 1 2 3 0 3 3
 2 0 2 1 3 0 1 3 3 1 2 2 2 3 1 2 3 3 0 0 1 2 1 0 1 0 0 0 2 0 2 2 1 1 2 3 1
 1 2 3 3 3 3 1 1 0 0 1 0 2 1 3 1 3 3 2 2 1 1 0 1 0 2 1 3 1 1 2 0 1 0 0 0 2
 2 0 0 2 2 0 1 3 1 2 1 3 3 1 2 2 0 1 1 1 1 3 0 1 3 3 2 1 3 1 3 0 1 1 0 0 3
 0 2 2 3 2 3 1 2 1 1 0 0 0 0 0 0 2 2 3 2 3 2 1 1 2 0 1 0 1 2 1 2 3 1 1 1 3
 0 1 0 3 2 2 2 2 2 0 0 2 3 3 3 1 2 3 3 1 1 1 0 0 3 0 2 2 3 2 3 3 3 1 0 2 2
 3 1 1 3 2 1 3 2 2 1 0 2 2 1 0 2 1 1 2 2 0 1 2 2 0 1 2 0 0 0 0 3 0 0 3 3 1
 1 0 2 0 0 1 3 0 0 0 1 2 0 0 1 1 1 1 2 2 0 2 3 1 2 1 2 3 0 2 1 1 2 3 0 1 0
 1 1 3 0 3 3 1 0 2 2 1 1 2 2 2 1 0 2 2 

In [13]:
#učitavanje modela sa google drive-a
test_model = keras.models.load_model("/content/drive/MyDrive/upgraded_trained_model")
print("Uspješno učitan model")

Successfuly loaded model


In [14]:
#testiranje modela i dohvaćanje izlaza modela
test_loss, test_acc = test_model.evaluate(int_test_ds)
print(f"Test accuracy: {test_acc:.3f}")

predictions = test_model.predict(int_test_ds)
print("Gotovo evaluiranje i predviđanje")

1958/1958 [==============================] - 93s 45ms/step - loss: 0.1953 - accuracy: 0.9310
Test accuracy: 0.931
1958/1958 [==============================] - 84s 42ms/step
Done evaluating and predicting


In [17]:
#većinsko glasanje i sastavljanje vektora za genom
final_vector=[]
for counter in range(len(predictions)):

  for i in range(korak):
    rez=0
    for j in range(int(data_length/korak)):

      if(predictions[counter-1*j][i+korak*j]>0.5):
        rez+=1
      else:
        rez+=0

    rez/=(data_length/korak)
    if(rez>0.5):
      final_vector.append(1)

    else:
      final_vector.append(0)


In [19]:
#funkcija za kreiranje anotacija
def dohvacanje_anotacija(izgladeni_vector):
  lista_anotacija=[]
  i=0
  ind=0
  first=True
  while(i<len(izgladeni_vector)):
    if(izgladeni_vector[i]==1 and first):
      ind=i
      first=False
      i+=1
    if(izgladeni_vector[i]==1 and not first):
      i+=1
      continue
    if(izgladeni_vector[i]==0 and not first):
      lista_anotacija.append([ind,i])
      first=True
      i+=1
    if(izgladeni_vector[i]==0 and first):
      i+=1
      continue

  print(lista_anotacija)
  return lista_anotacija


In [22]:
#zaglađivanje vektora kroz 5 iteracija
def izgladivanje(final_vector):

  izgladeni_vector=[]
  duljina_s_jedne_strane=10

  for baza in range(len(final_vector)):

    rez=0
    for i in range(1,duljina_s_jedne_strane+1):

        if(baza+i>=len(final_vector)):
          rez+=final_vector[baza+i-len(final_vector)] + final_vector[baza-i]
        else:
          rez+=(final_vector[baza+i] + final_vector[baza-i])

    rez=rez/(duljina_s_jedne_strane*2)

    if(rez>0.5):
      izgladeni_vector.append(1)
    else:
      izgladeni_vector.append(0)

  return izgladeni_vector

g=izgladivanje(final_vector)
for i in range(3):
  g=izgladivanje(g)

izgladeni_vector=izgladivanje(final_vector)
print("DONE")

6264500
6264500
6264500
6264500
6264500


In [24]:
#učitavanje anotacija iz gff datoteke
folder_path = "/content/drive/MyDrive/Test_gene_annotations"

gene_annotations = os.listdir(folder_path)

for bacteria_name in gene_annotations:

    folder_path="/content/drive/MyDrive" +"/Test_gene_annotations/"+bacteria_name+"/data"

    files_in_annotations = os.listdir(folder_path)

    for gca_file in files_in_annotations:


        if(gca_file.startswith("GCA")):
            folder_path="/content/drive/MyDrive"+"/Test_gene_annotations/"+bacteria_name+"/data/"+gca_file
            gca_files = os.listdir(folder_path)

            for file in gca_files:

                if(file=="genomic.gff"):
                    gff_file_path="/content/drive/MyDrive/"+"Test_gene_annotations/"+bacteria_name+"/data/"+gca_file+"/"+file


columns = ["seqid", "source", "type", "start", "end", "score", "strand", "phase", "attributes"]
GFF_DataFrame = pd.read_csv(gff_file_path, sep="\t", comment="#", names=columns)
print(GFF_DataFrame.head())



        seqid   source    type  start      end score strand phase  \
0  AE004091.2  Genbank  region      1  6264404     .      +     .   
1  AE004091.2  Genbank    gene    483     2027     .      +     .   
2  AE004091.2  Genbank     CDS    483     2027     .      +     0   
3  AE004091.2  Genbank    gene   2056     3159     .      +     .   
4  AE004091.2  Genbank     CDS   2056     3159     .      +     0   

                                          attributes  
0  ID=AE004091.2:1..6264404;Dbxref=taxon:208964;I...  
1  ID=gene-PA0001;Name=dnaA;gbkey=Gene;gene=dnaA;...  
2  ID=cds-AAG03391.1;Parent=gene-PA0001;Dbxref=NC...  
3  ID=gene-PA0002;Name=dnaN;gbkey=Gene;gene=dnaN;...  
4  ID=cds-AAG03392.1;Parent=gene-PA0002;Dbxref=NC...  


In [29]:
from inspect import EndOfBlock
gff_anotations=[]
for index,row in GFF_DataFrame.iterrows():
        if(row["type"]=="gene"):


            start_position = row["start"]
            end_position = row["end"]

            gff_anotations.append([start_position,end_position])

print("Anotacije GFF datoteke: ",gff_anotations)
print("Anotacije modela :",end=" ")
v=dohvacanje_anotacija(izgladeni_vector)

counter=0
error_margin=10
for row in gff_anotations:
  for my_row in v:
    if(abs(row[0]-my_row[0])<error_margin and abs(row[1]-my_row[1])<error_margin):
      counter+=1
print(counter,len(gff_anotations))
print("Final test accuracy: "+str(round(counter/len(gff_anotations)*100,4))+"%")

Anotacije GFF datoteke:  [[483, 2027], [2056, 3159], [3169, 4278], [4275, 6695], [7018, 7791], [7803, 8339], [8671, 10377], [10434, 12488], [12488, 13435], [13540, 14091], [14235, 15122], [15207, 15473], [15620, 16273], [16335, 16607], [16900, 17217], [17366, 18739], [18767, 20071], [20068, 21012], [21067, 21573], [21712, 22737], [22872, 23960], [24001, 24558], [24568, 25545], [25736, 26653], [26711, 27535], [27646, 28632], [28613, 29899], [29896, 30498], [30502, 32055], [32060, 32983], [33000, 34511], [34624, 35538], [35905, 36270], [36278, 36901], [37087, 37893], [37890, 39098], [39202, 40089], [40190, 40405], [40589, 40816], [41113, 42801], [42914, 53521], [56546, 56941], [57212, 58594], [58786, 60159], [60656, 61342], [61373, 61726], [61879, 62388], [62403, 62786], [63068, 64729], [65339, 65479], [66303, 68135], [68188, 68616], [69272, 69526], [69543, 70091], [70130, 70636], [70702, 71622], [71730, 72617], [72680, 73384], [73468, 73923], [74034, 74267], [74279, 74716], [74773, 7518